In [1]:
import numpy as np
import pandas as pd

In [4]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
%matplotlib tk

In [5]:
import os, os.path, sys

In [6]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\X"

In [7]:
dir = os.listdir(path)

Ucitavanje fajlova

In [8]:
file_names = []
for file in dir:
    file_names.append(file)

In [9]:
percent_train = 0.8

In [10]:
train_files_names = file_names[:int(percent_train*len(file_names))]
test_file_names = file_names[int(percent_train*len(file_names))::]

Ucitavanje zadate trajektorije

In [17]:
x_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\xCmd.txt",header=None).astype('int'))
y_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\yCmd.txt",header=None).astype('int'))

In [19]:
x_command_new = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\novi_deo\xCmd_novi_deo.txt",header=None).astype('int'))
y_command_new = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\novi_deo\yCmd_novi_deo.txt",header=None).astype('int'))

In [38]:
max(x_command)

array([180000])

In [45]:
plt.figure(figsize=(15,7))
plt.subplot(121)
plt.plot(x_command, y_command)
plt.axis([min(x_command), max(x_command), min(x_command), max(x_command)])

plt.subplot(122)
plt.plot(x_command_new, y_command_new)
plt.axis([min(x_command), max(x_command), min(x_command), max(x_command)])
plt.show()

Konkatenacija fajlova koji se posle pravilno rasporedjuju na adekvatne train/test skupove

In [12]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)
test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in test_file_names], axis=0)

Prethodno konkatenizovani podaci se adekvatno dele kako bi se izbeglo fiktivno stanje (stanje pri kraju rada i pocetku novog ako su konkatenizovani podaci)

In [13]:
for j in range(len(train_files_names)):
    exec(f"razlika_trening_{j} = []")
    for i in range(61000):
        exec(f"razlika_trening_{j}.append(x_command[i] - train_[j*61000+i])")
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')        

In [15]:
for j in range(len(test_file_names)):
    exec(f"razlika_test_{j} = []")
    for i in range(61000):
        exec(f"razlika_test_{j}.append(x_command[i] - test_[j*61000+i])")
    exec(f'razlika_test_{j} = np.asarray(razlika_test_{j})')    

Funkcija koja deli podatke na "sliding window"

In [14]:
def create_timeseries_split(data, time_steps):

  # Get the number of samples in the data
  num_samples = len(data) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data[i+time_steps, 0]

  return x_train, y_train

Velicina buffera

In [16]:
time_steps = 8

Deljenje podataka na train i test skup

In [17]:
for i in range(len(train_files_names)):
    exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, time_steps)')

In [18]:
for i in range(len(test_file_names)):
    exec(f'test_X_{i}, test_y_{i} = create_timeseries_split(razlika_test_{i}, time_steps)')

In [19]:
train_X = np.asarray(train_X_0)
for i in range(1,len(train_files_names)):
    exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

In [20]:
train_y = np.asarray(train_y_0)
for i in range(1,len(train_files_names)):
    exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')

In [21]:
test_X = np.asarray(test_X_0)
for i in range(1,len(test_file_names)):
    exec(f'test_X = np.append(test_X, test_X_{i}, axis=0)')

In [22]:
test_y = np.asarray(test_y_0)
for i in range(1,len(test_file_names)):
    exec(f'test_y = np.append(test_y, test_y_{i}, axis=0)')

In [45]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_0)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_0[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_0[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_0[j], color='red',marker='o',markersize=2)
    i += 1
plt.plot(x_command, linewidth = 1)
plt.show()

In [43]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_4)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_4[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_4[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_4[j], color='red',marker='o',markersize=2)
    i += 1
plt.plot(x_command, linewidth = 1)
plt.show()

In [41]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_3)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_3[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_3[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_3[j], color='red',marker='o',markersize=2)
    i += 1
plt.plot(x_command, linewidth = 1)
plt.show()

In [213]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_2)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_2[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_2[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_2[j], color='red',marker='o',markersize=2)
    i += 1
plt.plot(x_command, linewidth = 1)
plt.show()

In [212]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_1)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_1[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_1[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_1[j], color='red',marker='o',markersize=2)
    i += 1
plt.plot(x_command, linewidth = 1)
plt.show()

In [39]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_0)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_0[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_0[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_0[j], color='red',marker='o',markersize=2)
    i += 1
plt.plot(x_command, linewidth = 1)
plt.show()

In [38]:
min_trajektorija = []
max_trajektorija = []
for i in range(61000):
    min_trajektorija.append(x_command[i] + min_kord[i])
    max_trajektorija.append(x_command[i] + max_kord[i])

MIN MAX TRAJEKTORIJE

# NA TEST SKUPU - min/max plotovi

In [63]:
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [62]:
plt.subplot(211)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_5,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

plt.subplot(212)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_5,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [60]:
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_4,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [59]:
plt.subplot(211)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_4,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

plt.subplot(212)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_4,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [58]:
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_3,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [56]:
plt.subplot(211)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_3,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

plt.subplot(212)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_3,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [54]:
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

In [52]:
plt.subplot(211)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(test_[10:61000], 'g')
plt.show()

plt.subplot(212)
plt.plot(x_command[10::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(test_[10:61000], 'g')
plt.show()